In [1]:
import pandas as pd

In [2]:
header = ['userId', 'movieId', 'rating', 'timestamp']
ratings = pd.read_csv('./ml-100k/u.data', sep='\t', names=header)

u_unique = ratings.userId.unique()
user2Idx = {o:i for i,o in enumerate(u_unique)}

m_unique = ratings.movieId.unique()
movie2Idx = {o:i for i,o in enumerate(m_unique)}

ratings

FileNotFoundError: File b'./ml-100k/u.data' does not exist

In [31]:
ratings.userId = ratings.userId.apply(lambda x: user2Idx[x])
ratings.movieId = ratings.movieId.apply(lambda x: movie2Idx[x])

In [32]:
ratings

,userId,movieId,rating,timestamp
0,0,0,3,881250949
1,1,1,3,891717742
2,2,2,1,878887116
3,3,3,2,880606923
4,4,4,1,886397596
5,5,5,4,884182806
6,6,6,2,881171488
7,7,7,5,891628467
8,8,8,3,886324817
9,9,9,3,883603013


In [151]:
from sklearn.model_selection import train_test_split
ratings_train, ratings_val = train_test_split(ratings, test_size=0.2)

In [152]:
len(ratings_train), len(ratings_val)

(80000, 20000)

In [153]:
n_users = int(ratings.userId.nunique())
n_movies = int(ratings.movieId.nunique())
n_users_train = int(ratings_train.userId.nunique())
n_movies_train = int(ratings_train.movieId.nunique())
print(n_users, n_movies, n_users_train, n_movies_train)

943 1682 943 1647


In [154]:
from keras.layers import Embedding, Input, Flatten, Dot
from keras import Model
from keras.constraints import non_neg

In [158]:
n_latent_factors = 3

In [166]:
movie_input = Input(shape=[1], name='Item')
# movie_embedding = Embedding(n_movies + 1, n_latent_factors, name='Movie-Embedding', embeddings_constraint=non_neg())(movie_input)
movie_embedding = Embedding(n_movies + 1, n_latent_factors, name='Movie-Embedding')(movie_input)
movie_vec = Flatten(name='FlattenMovies')(movie_embedding)

user_input = Input(shape=[1],name='User')
user_vec = Flatten(name='FlattenUsers')(Embedding(n_users + 1, n_latent_factors,name='User-Embedding')(user_input))
#user_vec = Flatten(name='FlattenUsers')(Embedding(n_users + 1, 
#                                                  n_latent_factors,name='User-Embedding', embeddings_constraint=non_neg())(user_input))

In [167]:
prod = Dot(axes=1, name='DotProduct')([movie_vec, user_vec])
model = Model([user_input, movie_input], prod)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
Movie-Embedding (Embedding)     (None, 1, 3)         5049        Item[0][0]                       
__________________________________________________________________________________________________
User-Embedding (Embedding)      (None, 1, 3)         2832        User[0][0]                       
__________________________________________________________________________________________________
FlattenMov

In [168]:
from keras.optimizers import Adam

In [171]:
model.compile(Adam(lr=0.0001), 'mean_squared_error')

In [172]:
history = model.fit([ratings_train.userId, ratings_train.movieId], ratings_train.rating, 
                    validation_data=([ratings_val.userId, ratings_val.movieId], ratings_val.rating), epochs=100, verbose=1)

Train on 80000 samples, validate on 20000 samples
Epoch 1/100
80000/80000 [==============================] - 4s 55us/step - loss: 0.8127 - val_loss: 0.8899
Epoch 2/100
80000/80000 [==============================] - 4s 46us/step - loss: 0.8107 - val_loss: 0.8879
Epoch 3/100
80000/80000 [==============================] - 4s 45us/step - loss: 0.8091 - val_loss: 0.8864
Epoch 4/100
80000/80000 [==============================] - 4s 46us/step - loss: 0.8078 - val_loss: 0.8852
Epoch 5/100
80000/80000 [==============================] - 4s 47us/step - loss: 0.8068 - val_loss: 0.8843
Epoch 6/100
80000/80000 [==============================] - 4s 47us/step - loss: 0.8059 - val_loss: 0.8835
Epoch 7/100
80000/80000 [==============================] - 4s 47us/step - loss: 0.8050 - val_loss: 0.8829
Epoch 8/100
80000/80000 [==============================] - 4s 47us/step - loss: 0.8043 - val_loss: 0.8823
Epoch 9/100
80000/80000 [==============================] - 4s 48us/step - loss: 0.8036 - val_loss: 0.8